In [1]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from datetime import datetime
import pandas as pd
import os, re, time, json, random
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException


In [2]:
# starts browser
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.implicitly_wait(10)




====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\USER\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


In [3]:
mobile = '9910737626'
login_url = 'https://app.qandi.com/login'
mobile_email_xp = '//*[@id="mobile_num"]'
next_btn_xp = '//*[@id="mobile-input-btn"]'
enter_otp_xp = '//*[@id="otp_num"]'
verify_xp = '//*[@id="otp-verify-btn"]'
custom_url = 'https://app.qandi.com/exam_custom'

sub_xp = '//*[@id="Mathematics_btn"]'
save_and_next_xp_f = '//*[@id="question_section"]/div[3]/a[1]'
save_and_next_xp_o = '//*[@id="question_section"]/div/div[3]/a[1]'
q_type_xp = '//*[@id="question_section"]/div/div[1]/span'

wait = 3
prob = 1 # for toppers


In [4]:
q_data = pd.read_excel('all_live_questions.xlsx')
q_data[:1]

,question_id,class_id,subject_id,unit_id,chapter_id,topic_id,major_concept_id,difficulty_level,template_type,answers
0,2055,1,1,15.0,294.0,3470.0,2807.0,14.0,2.0,"{""3"":""<p><math xmlns=\""http:\/\/www.w3.org\/19..."


In [5]:
# Login to qandi

def login():
    driver.get(login_url)

    driver.find_element(By.ID, 'mobile_num').send_keys(mobile)
    next_btn_elm = driver.find_element(By.ID, 'mobile-input-btn')

    try:
        next_btn_elm.click()
        enter_otp_elm = driver.find_element(By.ID, 'otp_num')
    except:
        try:
            driver.execute_script("arguments[0].click();", next_btn_elm)
            enter_otp_elm = driver.find_element(By.ID, 'otp_num')
        except:
            print('Not able to login')

    otp = input('Please input otp and then press enter to continue:')
    enter_otp_elm.send_keys(otp)
    verify_elm = driver.find_element(By.ID, 'otp-verify-btn')

    try:
        verify_elm.click()
    except:
        driver.execute_script("arguments[0].click();", verify_elm)
    return
    

In [7]:
login()

Please input otp and then press enter to continue: 60619


In [8]:

def get_topic_ids(chp_id):
    chap_box_content = driver.find_element(By.ID, f'chapter_box_{chp_id}').get_attribute('innerHTML')
    topic_ids = re.findall(r'id="chpt_topic_(\d+)"', chap_box_content)
    topic_ids = [int(i) for i in topic_ids]
    return topic_ids


In [9]:

def get_curr_q_id():
    curr_q_xp = '//*[@id="current_question"]'
    curr_q_elm = driver.find_element(By.XPATH, curr_q_xp)
    curr_q_id = curr_q_elm.get_attribute('value')    
    return int(curr_q_id)
# get_curr_q_id()


In [10]:

def get_corr_options(q_id, prob):
    ans_text = q_data[q_data['question_id']==q_id]['answers'].values[0]
    ans_text_dict = json.loads(ans_text)
    corr_options = list(ans_text_dict.keys())
    corr_options = [int(i) for i in corr_options]
    incorr_options = [i for i in range(1,5) if i not in corr_options]
    if random.random() > prob:
        corr_options[0] = incorr_options[0]
    return corr_options


In [11]:

def do_question(o_xps, save_and_next_xp, wait, curr_q_id):
    # choose option
    for o_xp in o_xps:
        o_xp_elm = driver.find_element(By.XPATH, o_xp)
        time.sleep(wait)
        # o_xp_elm.click()
        try:
            driver.execute_script("arguments[0].click();", o_xp_elm)
        except:
            driver.execute_script("arguments[0].click();", o_xp_elm)
    
    # click save and next after wait time is over
    # time.sleep(wait)
    save_and_next_elm = driver.find_element(By.XPATH, save_and_next_xp)
    time.sleep(1)
    driver.execute_script("arguments[0].click();", save_and_next_elm)
    
    time.sleep(1)
    curr_id_temp = get_curr_q_id()
    # print(curr_id_temp, curr_q_id)
    if curr_id_temp == curr_q_id:
        time.sleep(1)
        driver.execute_script("arguments[0].click();", save_and_next_elm)
    if curr_id_temp == curr_q_id:
        time.sleep(1)
        driver.execute_script("arguments[0].click();", save_and_next_elm)
    return


In [12]:

def do_question_num(ans, save_and_next_xp, wait, curr_q_id):
    
    num_ans_box_xp = f'//*[@id="quest_option_{curr_q_id}"]'
    num_ans_box_elm = driver.find_element(By.XPATH, num_ans_box_xp)
    num_ans_box_elm.send_keys(ans)
    time.sleep(wait)
    save_and_next_elm = driver.find_element(By.XPATH, save_and_next_xp)
    time.sleep(1)
    driver.execute_script("arguments[0].click();", save_and_next_elm)
    

In [13]:
# Attempt Topic test

def do_topic_test():
    
    # Do first question
    curr_q_id = get_curr_q_id()
    # print(curr_q_id)
    q_type_elm = driver.find_element(By.XPATH, q_type_xp)
    q_type = q_type_elm.get_attribute('innerText')
    
    if q_type == 'NUMERICAL' or int(q_data[q_data['question_id']==curr_q_id]['template_type'].values[0])==11:
        ans = re.sub('[\[\]\"]', '', q_data[q_data['question_id']==curr_q_id]['answers'].values[0])
        do_question_num(ans, save_and_next_xp_f, wait, curr_q_id)
    else:
        corr_options = get_corr_options(curr_q_id, prob)
        count = 0
        o_xps = []
        for corr_option in corr_options:
            o_xp_f = f'//*[@id="question_section"]/div[2]/div[2]/div[{corr_option}]/div/label'
            o_xps.append(o_xp_f)

        do_question(o_xps, save_and_next_xp_f, wait, curr_q_id)
        
    count += 1
    # print(f'No. of questions done: {count}')

    # Do other questions
    sub_test_xp = '//*[@id="bt-modal-confirm_over"]'
    sub_test_elm = driver.find_element(By.XPATH, sub_test_xp)
    test_end_condition = sub_test_elm.get_attribute('innerText')

    while test_end_condition != 'SUBMIT TEST':
        
        curr_q_id = get_curr_q_id()
        # print(curr_q_id)
        q_type_elm = driver.find_element(By.XPATH, q_type_xp)
        q_type = q_type_elm.get_attribute('innerText')
        try:
            if q_type == 'NUMERICAL':
                ans = re.sub('[\[\]\"]', '', q_data[q_data['question_id']==curr_q_id]['answers'].values[0])
                do_question_num(ans, save_and_next_xp_o, wait, curr_q_id)
            else:
                o_xps = []
                # try:
                corr_options = get_corr_options(curr_q_id, prob)
                for corr_option in corr_options:
                    o_xp_o = f'//*[@id="question_section"]/div/div[2]/div[2]/div[{corr_option}]/div/label'
                    o_xps.append(o_xp_o)
                time.sleep(1)
                do_question(o_xps, save_and_next_xp_o, wait, curr_q_id)
        except StaleElementReferenceException:
            try:
                if q_type == 'NUMERICAL':
                    ans = re.sub('[\[\]\"]', '', q_data[q_data['question_id']==curr_q_id]['answers'].values[0])
                    do_question_num(ans, save_and_next_xp_o, wait, curr_q_id)
                else:
                    o_xps = []
                    # try:
                    corr_options = get_corr_options(curr_q_id, prob)
                    for corr_option in corr_options:
                        o_xp_o = f'//*[@id="question_section"]/div/div[2]/div[2]/div[{corr_option}]/div/label'
                        o_xps.append(o_xp_o)
                    time.sleep(1)
                    do_question(o_xps, save_and_next_xp_o, wait, curr_q_id)
            except StaleElementReferenceException:          
                if q_type == 'NUMERICAL':

                    input(f'Do the question with id: {curr_q_id} and ans as: {ans}')
                else:
                    input(f'Do the question with id: {curr_q_id} and correct option as as: {corr_options}')
        count += 1
        # print(f'No. of questions done: {count}')
        try:
            sub_test_elm = driver.find_element(By.XPATH, sub_test_xp)
            test_end_condition = sub_test_elm.get_attribute('innerText')
        except:
            break

    time.sleep(1)
    driver.execute_script("arguments[0].click();", sub_test_elm)
    # driver.save_screenshot(os.path.join(os.getcwd(),'botTestOutput',f'{Subject[0]}_{Topic}.png'))
    return 


In [37]:
# For Physics

# driver.get(custom_url)

# phy_list_innerHTML = driver.find_element(By.ID,'Physics_list').get_attribute('innerHTML')
# phy_chp_ids = re.findall(r'id="chapter_box_(\d+)"', phy_list_innerHTML)
# phy_chp_ids = [int(i) for i in phy_chp_ids]


In [14]:
driver.get(custom_url)

math_list_innerHTML = driver.find_element(By.ID,'Mathematics_list').get_attribute('innerHTML')
math_chp_ids = re.findall(r'id="chapter_box_(\d+)"', math_list_innerHTML)
math_chp_ids = [int(i) for i in math_chp_ids]


In [25]:
chp_no = 0

driver.get(custom_url)
for chp_id in math_chp_ids[12:]:
    # phy_tab_xp = '//*[@id="Physics_btn"]'
    # phy_tab_elm = driver.find_element(By.ID, "Physics_btn")
    # driver.execute_script("arguments[0].click();", phy_tab_elm)
    time.sleep(1)
    chp_expand_elm = driver.find_element(By.ID, f"expand_topic_{chp_id}")
    time.sleep(0.5)
    driver.execute_script("arguments[0].click();", chp_expand_elm)
    time.sleep(1)
    topic_ids = get_topic_ids(chp_id)
    
    topic_no = 0
    for topic_id in topic_ids:
        
        topic_select_elm = driver.find_element(By.ID, f"chpt_topic_{topic_id}")
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", topic_select_elm)
        take_test_for_sel_topic_elm = driver.find_element(By.XPATH, '//*[@id="topic_custom_footer"]/button')
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", take_test_for_sel_topic_elm)
        time.sleep(1)
        go_for_it_elm = driver.find_element(By.XPATH, '//*[@id="goto-exam-btn"]')
        test_instruction_elm = driver.find_element(By.ID, 'test_instruction')
        driver.execute_script("arguments[0].click();", go_for_it_elm)
        time.sleep(0.5)

        while test_instruction_elm.is_displayed():
            driver.execute_script("arguments[0].click();", go_for_it_elm)
            test_instruction_elm = driver.find_element(By.ID, 'test_instruction')
            print('At instruction page')
            time.sleep(0.5)
        
        try:
            do_topic_test()
        except :
            input('Please submit the exam.')
                
        time.sleep(1)
        driver.get(custom_url)
        time.sleep(1)
        chp_expand_elm = driver.find_element(By.ID, f"expand_topic_{chp_id}")
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", chp_expand_elm)
        time.sleep(1)
        topic_no+=1
        print(f'Topic completed: {topic_no}')
    
    chp_no+=1
    print(f'Chapters completed: {chp_no}')
    

Topic completed: 1
Chapters completed: 1
Topic completed: 1
Topic completed: 2
Topic completed: 3
Topic completed: 4
Topic completed: 5
Chapters completed: 2
Topic completed: 1
Topic completed: 2
Topic completed: 3
Topic completed: 4
Chapters completed: 3
Topic completed: 1
Topic completed: 2
Topic completed: 3
Chapters completed: 4
Topic completed: 1
Topic completed: 2
Topic completed: 3
Chapters completed: 5
Topic completed: 1
Topic completed: 2
Chapters completed: 6
Topic completed: 1
Topic completed: 2
Topic completed: 3
Topic completed: 4
Topic completed: 5
Topic completed: 6
Chapters completed: 7
Topic completed: 1
Topic completed: 2
Topic completed: 3
Topic completed: 4
Topic completed: 5
Topic completed: 6
Chapters completed: 8
Topic completed: 1
Topic completed: 2
Topic completed: 3
Topic completed: 4
Topic completed: 5
Topic completed: 6
Topic completed: 7
Topic completed: 8
Chapters completed: 9
Topic completed: 1
Topic completed: 2
Topic completed: 3
Topic completed: 4
Topi